In [142]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [143]:
torch.manual_seed(1)

## **Softmax**
숫자를 확률로 전환한다.
softmax function을 사용하는 이유
Activation Function을 통해서 Line Equation을 통해 생성된 Score 값을 어떤 확률로 변환하고 싶어한다.

우리가 계산하고자 하는 Output Classification의 개수 즉, Class의 개수가 여러개 일 때 사용하는 것이 바로 Softmax function으로 어떤 score값에 대해서 각 Class일 확률을 뽑아낼 수있다.

Socre 한개에 대해서 Class1일 확률, Class2일 확률, … ClassN일 확률이 나온다치면 모두 다 값이 0~1이어야하고 모두 다 더했을 때 당연히 1이 나와야한다.
ex) 고양이일 확률 70%, 강아지일 확률 20%, 코끼리일 확률 10%, 모두 다 더하면 100%

그런데 Score값은 앞서 말했듯이 음수도 나올 수 있으므로...

e를 씌워서 모두 양수 positive(+)로 만든다.
∑e로 나눠주어서 비율로 만들어준다.

In [144]:
z = torch.FloatTensor([1,2,3])

In [145]:
# Pytorch's softmax function.
hypothesis = F.softmax(z, dim=0)
print(hypothesis)

tensor([0.0900, 0.2447, 0.6652])


In [146]:
hypothesis.sum()

tensor(1.)

## **Cross Entropy Loss (Low-level)**

In [147]:
z = torch.rand(3,5, requires_grad = True)
hypothesis = F.softmax(z, dim=1)
print(hypothesis)

tensor([[0.2645, 0.1639, 0.1855, 0.2585, 0.1277],
        [0.2430, 0.1624, 0.2322, 0.1930, 0.1694],
        [0.2226, 0.1986, 0.2326, 0.1594, 0.1868]], grad_fn=<SoftmaxBackward0>)


In [148]:
y = torch.randint(5, (3,)).long()
print(y.unsqueeze(1))

tensor([[0],
        [2],
        [1]])


y.unsqueeeze(1)은 다음과 같다.

tensor([[0],
        [2],
        [1]])

y_one_hot.scatter_(1, y.unsqueeze(1),1)
이 코드는 y.unsqueeze(1)를 통해서 나온 index에 1을 채워 넣겠다는 의미

In [149]:
y_one_hot = torch.zeros_like(hypothesis)
print(y_one_hot)
print()
y_one_hot.scatter_(1, y.unsqueeze(1),1)
print(y_one_hot)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

tensor([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.]])


In [150]:
cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
print(cost)

tensor(1.4689, grad_fn=<MeanBackward0>)


## **Cross-entropy Loss with torch.nn.functional**

F.log_softmax()

F.nll_loss() : computes the negative loss likehood

F.cross_entropy(): combines F.log_softmax() and F.nll_loss()

In [151]:
#Low-level
torch.log(F.softmax(z, dim=1))

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]], grad_fn=<LogBackward0>)

In [152]:
# High Level
F.log_softmax(z, dim=1)

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]],
       grad_fn=<LogSoftmaxBackward0>)

In [153]:
# Low-level
(y_one_hot * -torch.log(F.softmax(z, dim=1))).sum(dim=1).mean()

tensor(1.4689, grad_fn=<MeanBackward0>)

In [154]:
# High Level
F.nll_loss(F.log_softmax(z,dim=1), y)

tensor(1.4689, grad_fn=<NllLossBackward0>)

In [155]:
F.cross_entropy(z,y)

tensor(1.4689, grad_fn=<NllLossBackward0>)

## **Training with Low-level Cross Entropy Loss**

In [156]:
x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
y_train = [2, 2, 2, 1, 1, 1, 0, 0]
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

In [157]:
W = torch.zeros((4,3), requires_grad = True)
b = torch.zeros(1, requires_grad = True)

optimizer = optim.SGD([W,b], lr = 0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):


  #Cost계산
  hypothesis = F.softmax(x_train.mm(W) + b, dim =1)
  y_one_hot = torch.zeros_like(hypothesis)
  y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
  cost = (y_one_hot * -torch.log(F.softmax(hypothesis, dim=1))).sum(dim=1).mean()

  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  # 100번마다 로그 출력
  if epoch % 100 == 0:
      print('Epoch {:4d}/{} Cost: {:.6f}'.format(
          epoch, nb_epochs, cost.item()
      ))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.901535
Epoch  200/1000 Cost: 0.839114
Epoch  300/1000 Cost: 0.807826
Epoch  400/1000 Cost: 0.788472
Epoch  500/1000 Cost: 0.774822
Epoch  600/1000 Cost: 0.764449
Epoch  700/1000 Cost: 0.756191
Epoch  800/1000 Cost: 0.749398
Epoch  900/1000 Cost: 0.743671
Epoch 1000/1000 Cost: 0.738749


## **Training with F.cross_entropy**

In [158]:
W = torch.zeros((4,3), requires_grad = True)
b = torch.zeros(1, requires_grad=True)

optimizer = optim.SGD([W,b], lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

  hypothesis = x_train.mm(W) + b
  cost = F.cross_entropy(hypothesis, y_train)

  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  if epoch % 100 == 0:
    print('Epoch {:4d}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, cost.item()
    ))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.761050
Epoch  200/1000 Cost: 0.689991
Epoch  300/1000 Cost: 0.643229
Epoch  400/1000 Cost: 0.604117
Epoch  500/1000 Cost: 0.568255
Epoch  600/1000 Cost: 0.533922
Epoch  700/1000 Cost: 0.500291
Epoch  800/1000 Cost: 0.466908
Epoch  900/1000 Cost: 0.433507
Epoch 1000/1000 Cost: 0.399962


## **High-level Implementation with nn.Module**

In [159]:
class SoftmaxClassifierModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear = nn.Linear(4,3)

  def forward(self, x):
    return self.linear(x)

In [160]:
model = SoftmaxClassifierModel()

In [161]:
optimizer = optim.SGD(model.parameters(), lr = 0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

  prediction = model(x_train)

  cost = F.cross_entropy(prediction, y_train)

  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  if epoch % 100 == 0:
      print('Epoch {:4d}/{} Cost: {:.6f}'.format(
          epoch, nb_epochs, cost.item()
      ))

Epoch    0/1000 Cost: 1.849513
Epoch  100/1000 Cost: 0.689894
Epoch  200/1000 Cost: 0.609259
Epoch  300/1000 Cost: 0.551218
Epoch  400/1000 Cost: 0.500141
Epoch  500/1000 Cost: 0.451947
Epoch  600/1000 Cost: 0.405051
Epoch  700/1000 Cost: 0.358733
Epoch  800/1000 Cost: 0.312912
Epoch  900/1000 Cost: 0.269522
Epoch 1000/1000 Cost: 0.241922
